# Importações

In [1]:
import tensorflow as tf
import requests
import pandas as pd
import numpy as np

import json
import math

2023-07-15 13:10:34.949642: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-15 13:10:37.930001: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-07-15 13:10:37.930034: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-07-15 13:10:45.784343: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

# Obtendo Experimentos

In [9]:
url = "http://vps-40d69db1.vps.ovh.ca:23004/api/ExperimentoPreRedeTreinadas/listaParaScore"

r = requests.get(url)
j = r.json()
dfExperimento = pd.DataFrame.from_dict(j)
dfExperimento.head()

,id,nome,experimentoPreRedeId,grupoAcaoId,regraProjecaoId,geraScore,periodoTreinoRedeId,nomeArquivoRede
0,1,01,1,12,127,S,7,ExperimentoSintetico_Prometeu_v0_aInicial


# Globais

In [10]:
global experimentoCorrente
global diaCorrente
global modelo
global dfDados
global X

# Trata Experimento

In [11]:
def trataExperimento(experimento):
    experimentoCorrente = treino
    nome_arquivo = '/root/executa-rede/redes/' + experimento.nomeArquivoRede + '.h5'
    # carrega modelo
    modelo = tf.keras.models.load_model(nome_arquivo)
    urlDias = "http://vps-40d69db1.vps.ovh.ca:23004/api/DiaPregaos/obtemPeriodoPrevisaoTeste?idPeriodoTreinoRede={}".format(experimento.periodoTreinoRedeId)
    r = requests.get(urlDias)
    j = r.json()
    # carrega dfDias
    dfDias = pd.DataFrame.from_dict(j)
    print(dfDias.head())
    dfDias.apply(trataDiaPregao, axis='columns')

# Trata Dia

In [ ]:
def trataDiaPregao(dia):
    print('Tratando Dia: ' , dia.diaNum)
    diaCorrente = dia.diaNum
    urlExemplos = "http://vps-40d69db1.vps.ovh.ca:23004/api/ExemploTreinoAcaoEntradas/obtemPorDiaNum?diaNum={}".format(dia.diaNum)
    print(urlExemplos)
    r = requests.get(urlExemplos)
    j = r.json()
    # meu dados totais de entrada
    dfDados = pd.DataFrame.from_dict(j)
    #print(dfDados.head())
    dfTratadoX = Y.apply(lambda linha: np.fromstring(linha['campoX'], dtype=float, sep=','), axis='columns')
    # meu dataframe X
    X = np.stack(dfTratadoX.values)
    X.shape
    #dfDados.drop('campoX', inplace=True, axis=1)
    executaPrevisao()

# Cria as previsões

In [ ]:
def executaPrevisao():
    dfPrevisao = Y
    result = modelo.predict(X)
    dfPrevisao['score'] = result
    dfPrevisao['experimentoId'] = str(experimentoCorrente['id'])
    dfPrevisao.drop('campoY', inplace=True, axis=1)
    urlPrevisao = 'http://vps-40d69db1.vps.ovh.ca:23004/api/ExperimentoPreRedeResultados/recebePrevisaoTeste'
    print(urlPrevisao)
    dfPrevisao = pd.merge(dfDados,dfEntradas, how='left', left_on='ticker' ,  right_on='ticker')
    #print('ticker previsão:' , len(dfPrevisao))
    json = dfPrevisao.to_dict(orient='list')
    urlPrevisao = 'http://vps-40d69db1.vps.ovh.ca:23004/api/PrevisaoTestes/recebePrevisaoTreinoTeste'
    #print(json)
    response = requests.post(urlPrevisao, data=json)
    print(response)